In [5]:
import torch

from recommendation_engine.configs.config_sbert_mps import config as sbert_config
from recommendation_engine.configs.config_review_meta_score import config as rm_config
from recommendation_engine.models.dual_encoder_sbert import DualEncoderSBERT
from recommendation_engine.models.review_metadata_score_model import ReviewMetadataScoreModel


In [25]:
def export_dual_encoder_branch(checkpoint_path: str, output_dir: str, output_filename: str, config):
    """
    Exports one branch of DualEncoderSBERT (view1 or view2) to TorchScript format.
    """
    model = DualEncoderSBERT.from_config(config)
    model.load_state_dict(torch.load(checkpoint_path, map_location="cpu"))
    model.eval()

    batch_size = 1
    max_length = config["max_length"]
    dummy_input_ids = torch.randint(0, 100, (batch_size, max_length))
    dummy_attention_mask = torch.ones((batch_size, max_length), dtype=torch.long)

    ts_model = torch.jit.trace(model, (dummy_input_ids, dummy_attention_mask))
    output_path = f"{output_dir}/{output_filename}"
    torch.jit.save(ts_model, output_path)
    print(f"DualEncoderSBERT branch exported to TorchScript at: {output_path}")


In [24]:
def export_meta_review_model(checkpoint_path: str, output_dir: str, output_filename: str, config):
    """
    Exports ReviewMetadataScoreModel to TorchScript format.
    """
    model = ReviewMetadataScoreModel.from_config(config)
    model.load_state_dict(torch.load(checkpoint_path, map_location="cpu"))
    model.eval()

    batch_size = 1
    max_length = config["max_length"]

    dummy_review_input_ids = torch.randint(0, 100, (batch_size, max_length))
    dummy_review_attention_mask = torch.ones((batch_size, max_length), dtype=torch.long)
    dummy_meta_input_ids = torch.randint(0, config["meta_vocab_size"], (batch_size, max_length))
    dummy_meta_attention_mask = torch.ones((batch_size, max_length), dtype=torch.long)
    dummy_price = torch.tensor([10.0], dtype=torch.float)

    ts_model = torch.jit.trace(
        model,
        (dummy_review_input_ids, dummy_review_attention_mask,
         dummy_meta_input_ids, dummy_meta_attention_mask,
         dummy_price)
    )
    output_path = f"{output_dir}/{output_filename}"
    torch.jit.save(ts_model, output_path)
    print(f"ReviewMetadataScoreModel exported to TorchScript at: {output_path}")


In [26]:
# view1
export_dual_encoder_branch(
        checkpoint_path="/Users/sanamoin/Documents/sites/gadgets/recommendation_engine/outputs/sbert_nvidia/run_20250331_224539/checkpoints/best_model_view1.pt",
        output_dir="/Users/sanamoin/Documents/sites/gadgets/recommendation_engine/outputs/ts/",
        output_filename="dual_encoder_sbert_view1_ts.pt",
        config=sbert_config
    )

# view2
export_dual_encoder_branch(
    checkpoint_path="/Users/sanamoin/Documents/sites/gadgets/recommendation_engine/outputs/sbert_nvidia/run_20250331_224539/checkpoints/best_model_view2.pt",
    output_dir="/Users/sanamoin/Documents/sites/gadgets/recommendation_engine/outputs/ts/",
    output_filename="dual_encoder_sbert_view2_ts.pt",
    config=sbert_config
)

DualEncoderSBERT branch exported to TorchScript at: /Users/sanamoin/Documents/sites/gadgets/recommendation_engine/outputs/ts//dual_encoder_sbert_view1_ts.pt
DualEncoderSBERT branch exported to TorchScript at: /Users/sanamoin/Documents/sites/gadgets/recommendation_engine/outputs/ts//dual_encoder_sbert_view2_ts.pt


In [23]:
export_meta_review_model(
    checkpoint_path="/Users/sanamoin/Documents/sites/gadgets/recommendation_engine/outputs/meta_review_regressor/run_20250202_233100/checkpoints/best_model_ep4.pth",
    output_dir="/Users/sanamoin/Documents/sites/gadgets/recommendation_engine/outputs/ts/",
    output_filename="review_metadata_score_model_ts.pt",
    config=rm_config
)

Review Meta  exported to TorchScript at: /Users/sanamoin/Documents/sites/gadgets/recommendation_engine/outputs/ts/
